In [1]:
from bs4 import BeautifulSoup
import requests

competition_urls = {
	'football':
	{
		"ligue1": "https://www.zebet.fr/fr/competition/96-ligue_1_uber_eats",
		"liga": "https://www.zebet.fr/fr/competition/306-laliga",
		"bundesliga": "https://www.zebet.fr/fr/competition/268-bundesliga",
		"premier-league": "https://www.zebet.fr/fr/competition/94-premier_league",
		"serie-a": "https://www.zebet.fr/fr/competition/305-serie_a",
		"primeira": "https://www.zebet.fr/fr/competition/154-primeira_liga",
		"serie-a-brasil": "https://www.zebet.fr/fr/competition/81-brasileirao",
		"a-league": "https://www.zebet.fr/fr/competition/2169-a_league",
		"bundesliga-austria": "https://www.zebet.fr/fr/competition/131-bundesliga",
		"division-1a": "https://www.zebet.fr/fr/competition/101-pro_league_1a",
		"super-lig": "https://www.zebet.fr/fr/competition/254-super_lig",
	},
	'basketball':
	{
		"nba": "https://www.zebet.fr/fr/competition/206-nba",
		"euroleague": "https://www.zebet.fr/fr/competition/12044-euroligue",
	}
}

def get_page(competition):
	if (competition["sport"] in competition_urls and competition["competition"] in competition_urls[competition["sport"]]):
		url = competition_urls[competition["sport"]][competition["competition"]]
	else:
		return None
	response = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"})
	html = BeautifulSoup(response.content, 'html.parser')
	return html

def get_games(competition):
	html = get_page(competition)
	games = []
	game_elements = html.select(".pari-1")
	for el in game_elements:
		names = el.select(".pmq-cote-acteur")
		team1 = "".join(names[0].text.split())
		if (competition["sport"] == "football"):
			team2 = "".join(names[4].text.split())
		elif (competition["sport"] == "basketball"):
			team2 = "".join(names[2].text.split())
		odd_els = el.select(".pmq-cote")
		odds = []
		for odd_el in odd_els[::2]:
			odds.append(float(odd_el.text.replace(",", ".")))
		games.append({
			'team1': team1,
			'team2': team2,
			'odds': odds
		})
	return games

In [2]:
# maintenant on souhaite manipuler des dataframes : 
import pandas as pd 
import numpy as np 
def into_a_pd(competition):
    games = get_games(competition=competition)
    team1 = []
    team2 = []
    odd1 = []
    odd2 = []
    oddnul = []
    if len(games[0]['odds']) == 3 :
        for i in range(len(games)) :
            team1.append(games[i]['team1'])
            team2.append(games[i]['team2'])
            odd1.append(games[i]['odds'][0])
            oddnul.append(games[i]['odds'][1])
            odd2.append(games[i]['odds'][2]) 

            data = {
                'team1' : team1,
                'team2' : team2,
                'odd of team 1 winning' : odd1,
                'odds of equality ': oddnul,
                'odd of team 2 winning' : odd2,
            }
    else :
        for i in range(len(games)) :
            team1.append(games[i]['team1'])
            team2.append(games[i]['team2'])
            odd1.append(games[i]['odds'][0])
            odd2.append(games[i]['odds'][1])
        data = {
                'team1' : team1,
                'team2' : team2,
                'odd of team 1 winning' : odd1,
                'odd of team 2 winning' : odd2,
            }
   

    df = pd.DataFrame(data=data)

    return df 

into_a_pd(competition = {'sport': 'football', 'competition': 'ligue1'})

,team1,team2,odd of team 1 winning,odds of equality,odd of team 2 winning
0,Reims,Rennes,3.45,3.35,2.10
1,Nice,Lens,3.05,2.95,2.50
2,Marseille,Toulouse,1.43,4.80,6.10
3,Angers,Lorient,2.67,3.10,2.27
4,Nantes,Auxerre,1.80,3.25,3.65
5,Monaco,Brest,1.34,4.50,5.90
6,Toulouse,ACAjaccio,1.85,3.15,3.55
7,Lyon,Clermont,1.28,4.80,6.75
8,Lens,ParisSG,3.05,3.30,1.97
9,Strasbourg,Troyes,1.63,3.55,4.10
